# Mas APIS DE EQUITY

# Ameritrade

Para acceder a la API de TDA el broker ameritrade, hay que ingresar a https://developer.tdameritrade.com/
    
Y obviamente sacarse una cuenta de usuario gratuita, yo saqué la mia

Una vez creada la cuenta tienen que crear una app para que le den una "Consumer Key" similar a esta:

In [4]:
c_key = 'WKUEAMWXTPPTGLG8U3QTHW1TGEWAT214'

Una vez obtenida la key pueden usarla para llamar a la API

Obviamente hay que importar requests

In [5]:
import requests

Y ahora si probamos algún endpoint

In [6]:
def getQuote(symbol):
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/'+symbol+'/quotes?'
    r = requests.get(url=endpoint ,params={'apikey' : c_key})
    return r.json()

getQuote('AAPL')

{'AAPL': {'assetType': 'EQUITY',
  'assetMainType': 'EQUITY',
  'cusip': '037833100',
  'symbol': 'AAPL',
  'description': 'Apple Inc. - Common Stock',
  'bidPrice': 114.77,
  'bidSize': 300,
  'bidId': 'P',
  'askPrice': 114.78,
  'askSize': 1200,
  'askId': 'P',
  'lastPrice': 114.77,
  'lastSize': 0,
  'lastId': 'P',
  'openPrice': 118.33,
  'highPrice': 118.829,
  'lowPrice': 113.61,
  'bidTick': ' ',
  'closePrice': 115.355,
  'netChange': -0.585,
  'totalVolume': 184642039,
  'quoteTimeInLong': 1600214399804,
  'tradeTimeInLong': 1600214399804,
  'mark': 115.54,
  'exchange': 'q',
  'exchangeName': 'NASD',
  'marginable': True,
  'shortable': True,
  'volatility': 0.0157,
  'digits': 4,
  '52WkHigh': 137.98,
  '52WkLow': 53.1525,
  'nAV': 0.0,
  'peRatio': 34.0497,
  'divAmount': 0.82,
  'divYield': 0.71,
  'divDate': '2020-08-07 00:00:00.000',
  'securityStatus': 'Normal',
  'regularMarketLastPrice': 115.54,
  'regularMarketLastSize': 49773,
  'regularMarketNetChange': 0.185,
  

### Precios históricos

* Periodos válidos para cada periodType (los del * son los default)
    * day: 1, 2, 3, 4, 5, 10*
    * month: 1*, 2, 3, 6
    * year: 1*, 2, 3, 5, 10, 15, 20
    * ytd: 1*

* FrequencyType válidos para cada periodType (los del * son los default)
    * day: minute*
    * month: daily, weekly*
    * year: daily, weekly, monthly*
    * ytd: daily, weekly*
    
* Frequency válidos para cada FrequencyType (los del * son los default)
    * minute: 1*, 5, 10, 15, 30
    * daily: 1*
    * weekly: 1*
    * monthly: 1*

Otros parámetros posibles:
    * startDate: timestamp milisegundos desde epoch
    * endDate: timestamp milisegundos desde epoch

In [62]:
import pandas as pd

def getHist(symbol, periodT='day', period=1, freqT='minute', freq=1, na='true'):
    
    params = {'symbol':symbol, 'periodType':periodT, 'period':period, 
              'apikey' : c_key, 'frequencyType':freqT, 'frequency':freq ,
              'needExtendedHoursData':na}
    
    url_base = 'https://api.tdameritrade.com/v1/marketdata/'
    endpoint = url_base + symbol+'/pricehistory'
    
    r = requests.get(url=endpoint ,params=params)
    try:
        js = r.json()['candles']
        df = pd.DataFrame(js)
        df['fecha'] = pd.to_datetime(df.datetime, unit='ms')
    except:
        print('Error, ojo que tenemos una rta inesperada', r.json())
        df = pd.DataFrame()
        
    return df

data = getHist('AAPL', periodT='day', period=1, freqT='minute', freq=1)
data

,open,high,low,close,volume,datetime,fecha
0,117.51,117.59,117.02,117.38,33987,1600167600000,2020-09-15 11:00:00
1,117.32,117.40,117.25,117.25,10469,1600167660000,2020-09-15 11:01:00
2,117.29,117.32,117.00,117.00,25836,1600167720000,2020-09-15 11:02:00
3,117.01,117.25,117.00,117.00,13041,1600167780000,2020-09-15 11:03:00
4,117.01,117.20,117.01,117.20,4252,1600167840000,2020-09-15 11:04:00
...,...,...,...,...,...,...,...
775,114.70,114.72,114.69,114.70,4258,1600214100000,2020-09-15 23:55:00
776,114.68,114.75,114.68,114.74,16808,1600214160000,2020-09-15 23:56:00
777,114.74,114.76,114.73,114.75,8262,1600214220000,2020-09-15 23:57:00
778,114.77,114.79,114.73,114.75,12904,1600214280000,2020-09-15 23:58:00


Como verán en este caso usamos un try/except por si pasamos mal algun parámetro pero que ademas del mensaje de error me devuelva igual el dataFrame vacio

Esto es una buena práctica ppara evitar cortes del flujo del script inesperados

### Fundamentals

In [6]:
def fundamentals(symbol, pr='fundamental'):
    
    params = {'apikey' : c_key, 'symbol':symbol, 'projection': pr}
    
    endpoint = 'https://api.tdameritrade.com/v1/instruments'
    
    r = requests.get(url=endpoint ,params=params)
    return r.json()

fundamentals('AAPL')

{'AAPL': {'fundamental': {'symbol': 'AAPL',
   'high52': 399.82,
   'low52': 192.58,
   'dividendAmount': 3.28,
   'dividendYield': 0.84,
   'dividendDate': '2020-05-08 00:00:00.000',
   'peRatio': 30.50251,
   'pegRatio': 3.268477,
   'pbRatio': 21.4526,
   'prRatio': 6.29316,
   'pcfRatio': 24.46643,
   'grossMarginTTM': 38.10979,
   'grossMarginMRQ': 38.36194,
   'netProfitMarginTTM': 21.35039,
   'netProfitMarginMRQ': 19.29072,
   'operatingMarginTTM': 24.47599,
   'operatingMarginMRQ': 22.0414,
   'returnOnEquity': 62.09404,
   'returnOnAssets': 17.27511,
   'returnOnInvestment': 24.21635,
   'quickRatio': 1.46127,
   'currentRatio': 1.49596,
   'interestCoverage': 0.0,
   'totalDebtToCapital': 58.41288,
   'ltDebtToEquity': 114.3959,
   'totalDebtToEquity': 140.459,
   'epsTTM': 12.756,
   'epsChangePercentTTM': 9.33233,
   'epsChangeYear': 3.83907,
   'epsChange': 0.0,
   'revChangeYear': 0.0,
   'revChangeTTM': 3.67171,
   'revChangeIn': 0.0,
   'sharesOutstanding': 4334335000.

### Cadena de Opciones

Este es el endpoint mas jugoso de la API a mi entender ya que es un dato de mercado dificil de conseguir en  formma gratuita y fidedigna esto de las cadenas de opciones

Les dejo la implementación en python de un llamado al endpoint de opciones sencillo que me devuelve en principio todo un json y como es muy compleja la estructura vamos trabajandola de a poco luego

In [7]:
import requests

def options(symbol):
    
    params = {'apikey' : c_key, 'symbol':symbol}
    
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/chains'
    
    r = requests.get(url=endpoint ,params=params)
    return r.json()

chain = options('GGAL')

Lo que me devuelve es una estructura mas o menos asi:


{'symbol': 'GGAL',<br>
 'status': 'SUCCESS',<br>
 'underlying': None,<br>
 'strategy': 'SINGLE',<br>
 'interval': 0.0,<br>
 'isDelayed': True,<br>
 'isIndex': False,<br>
 'interestRate': 0.1,<br>
 'underlyingPrice': 11.41,<br>
 'volatility': 29.0,<br>
 'daysToExpiration': 0.0,<br>
 'numberOfContracts': 120,<br>
 'putExpDateMap': {'2020-07-17:0': {'2.5': [.. y aca todas las claves de ese strike de ese vencimiento de ese side(put)
 
 ... y asi los (en este caso 60) puts, y luego lo mismo para los 60 calls

Como esa estructura es demasiado rebuscada para listar, calcular, graficar, etc, vamos mejor a crear listados de fechas de vencimientos y de contratos por un lado y luego vamos a ir metiendolo en un dataFrame

In [8]:
v_calls = list(chain['callExpDateMap'].values())
v_calls_fechas = list(chain['callExpDateMap'].keys())
v_puts = list(chain['putExpDateMap'].values())
v_puts_fechas = list(chain['putExpDateMap'].keys())

print(v_calls_fechas)

['2020-08-21:28', '2020-09-18:56', '2020-10-16:84', '2021-01-15:175', '2022-01-21:546']


Bueno, como vemos ahi ya tenemos los vencimientos y los contratos, ahora resta recorrer esas listas y meter la data en un frame

In [9]:
v_calls = list(chain['callExpDateMap'].values())
v_calls_fechas = list(chain['callExpDateMap'].keys())
v_puts = list(chain['putExpDateMap'].values())
v_puts_fechas = list(chain['putExpDateMap'].keys())

calls = []
for i in range(len(v_calls)):
    v = list(v_calls[i].values())    
    for j in range(len(v)):
        calls.append(v[j][0])

puts = []
for i in range(len(v_puts)):
    v = list(v_puts[i].values())    
    for j in range(len(v)):
        puts.append(v[j][0])
        
contracts = pd.concat([pd.DataFrame(calls),pd.DataFrame(puts)])
contracts.columns

Index(['putCall', 'symbol', 'description', 'exchangeName', 'bid', 'ask',
       'last', 'mark', 'bidSize', 'askSize', 'bidAskSize', 'lastSize',
       'highPrice', 'lowPrice', 'openPrice', 'closePrice', 'totalVolume',
       'tradeDate', 'tradeTimeInLong', 'quoteTimeInLong', 'netChange',
       'volatility', 'delta', 'gamma', 'theta', 'vega', 'rho', 'openInterest',
       'timeValue', 'theoreticalOptionValue', 'theoreticalVolatility',
       'optionDeliverablesList', 'strikePrice', 'expirationDate',
       'daysToExpiration', 'expirationType', 'lastTradingDay', 'multiplier',
       'settlementType', 'deliverableNote', 'isIndexOption', 'percentChange',
       'markChange', 'markPercentChange', 'inTheMoney', 'nonStandard', 'mini'],
      dtype='object')

Como la cantidad de columnas es muy grande, armamos una tabla cualquiera para mostrar mas columnas

In [30]:
tabla = contracts.loc[contracts.daysToExpiration>0]
tabla = tabla.loc[:,['strikePrice','daysToExpiration','putCall','bid','ask',
                     'last','volatility','openInterest',
                     'theoreticalOptionValue']]

tabla.columns = ['Strike','Dias','Tipo','Bid','Ask',
                  'Ultimo','VI','OpenInt','PrimaT']
tabla

,Strike,Dias,Tipo,Bid,Ask,Ultimo,VI,OpenInt,PrimaT
0,2.5,28,CALL,9.80,10.80,0.00,488.089,0,10.300
1,5.0,28,CALL,7.10,8.00,6.30,218.348,3,7.550
2,7.5,28,CALL,4.80,5.90,4.50,186.005,27,5.350
3,10.0,28,CALL,2.55,2.95,3.10,94.005,1522,2.750
4,12.5,28,CALL,0.95,1.05,1.05,74.890,2204,1.000
...,...,...,...,...,...,...,...,...,...
46,12.5,546,PUT,2.25,5.50,4.06,65.255,17,3.875
47,15.0,546,PUT,3.00,7.40,7.53,58.479,17,5.200
48,17.5,546,PUT,5.00,9.40,10.55,60.628,1,7.200
49,20.0,546,PUT,7.00,11.40,9.40,60.788,52,9.200


# Otras

* MarketStack
* Finnhub
* IEX



* ROFEX
* BYMA- IOL
* ALPACA

# Tiingo